#Student Name:Salih Eren Yüzbaşıoğlu Studen ID:2220356040

KNN Learning Algorithm 

In this assignment we are given a data set containing 60000 rows of information about 60 questions that were asked to 
60000 people and their answer scaling from -3 to 3 and at the end their character found by doctors. Our duty is to make an algorithm to guess people's charecters by looking at the questions that were asked to them by using K nearest algoritm

In [1]:
import pandas as pd
import numpy as np
import time
def read_input(data_to_read):
    data=pd.read_csv(data_to_read,encoding='latin1')
    data = data.sample(frac = 1)
    data=data.iloc[:,1:]

    data["Personality"]=(data["Personality"]=="ESTJ")*0 | (data["Personality"]=="ENTJ")*1 | (data["Personality"]=="ESFJ")*2 | (data["Personality"]=="ENFJ")*3 |(data["Personality"]=="ISTJ")*4 | (data["Personality"]=="ISFJ")*5 | (data["Personality"]=="INTJ")*6 | (data["Personality"]=="INFJ")*7 | (data["Personality"]=="ESTP")*8 | (data["Personality"]=="ESFP")*9 | (data["Personality"]=="ENTP")*10 | (data["Personality"]=="ENFP")*11 | (data["Personality"]=="ISTP")*12 | (data["Personality"]=="ISFP")*13 | (data["Personality"]=="INTP")*14 |(data["Personality"]=="INFP")*15 
    data=data.astype(int)
    return data


Here we import the library that we were supposed to use. Read_input basically reads the given excell file and turns it into pandas DataFrame then it shuffles the data and deletes the bir column since it is response id useless for our project. Then gives integer values for ever personality and turns the whole data to integer values.

In [2]:
def normalize(data):
    normalized_data=(data+3)/6
    normalized_data["Personality"]=data["Personality"]
    return normalized_data


This is not in read_input because we will do both normalized and not normalized.

In [7]:

def Knn(k,whole_info_el,k_amount,data,len_Data):
    Guess_amount_list=np.zeros([16,16], dtype = int)
    Wrong_guesses=[]
    test_len=len_Data//k_amount
    
    all_indexes=[(test_len*constant) for constant in range(1,k_amount+1)]
    percen_constant=(100/test_len)/k_amount

    percentage=float(0)
    for cross in range(k_amount):

        learn1=data.iloc[:all_indexes[cross]-test_len]
        learn2=+data.iloc[all_indexes[cross]:]
        combine=[learn1,learn2]
        learn=np.array(pd.concat(combine))

        test=np.array(data.iloc[all_indexes[cross]-test_len:all_indexes[cross]])       
        
        for line in range(test_len):

            percentage+=percen_constant
            print(f"completed: {int(percentage)}%",end='\r')


            difference=learn[:,:-1]-test[line,:-1]

            distance=np.einsum('ij,ij->i',difference,difference)

            smallest_k=np.argpartition(distance, k)[:k]
            most_present=np.argmax(np.bincount(learn[ smallest_k,-1].astype(int)))


            if int(test[line,-1])!=most_present:
                Wrong_guesses.append([int(test[line,-1]),most_present,smallest_k])

            Guess_amount_list[int(test[line,-1]),most_present]+=1
    whole_info_el["Wrong_guesses"]=Wrong_guesses
    whole_info_el["Confusion_list"]=Guess_amount_list

    return Guess_amount_list

This is the man in KNN function that takes data , a fairly empty dictionary to fill the results with , lenght of the data , k_amount corresponding the k in k-cross validation and k corresponding the k in KNN. Function starts by creating confusion matrix named Guess_amount_list this matrix will later be added to dictionary after we fill it completely, function also creates wrong_guesses list that will contain every wrong guess which also will be added to dictionary and will be used to determine why the guesses were wrong and investigation. After that we create a list containg the indexes that we wants cut the data in order for this function to be flexible with other datas I made it so it checks data lenght and divides it by k_amount . There is also percen_constant which is just a constant that we add to percentage every loop so and print percentage to know how much works is left. After that we have a for that turns for range of k_amount so in this example it turns 5 times for 5 cross validation, in for we see the defining of the learn and test arrays after that there is for loop for every line in test in for loop we print the percentage and make sure it is '\r' so we see it live after that we find the difference between learn array and test array's line then to find distance we find np.einsum('ij,ij->i',difference,difference) which basically takes the square of every line in difference array and sums every line donated as j in a very efficient way. Then we use argpartition to find the smallest k elements which is also very efficient because it only does not spend time sorting the whole array but just finds smallest k amount then by combining bincount and argmax we first find how many times a value has been used with bincount then using argmax we find the biggest of them after that we check our guess and the real value if it is different we add it to wrong_guesses and also we upgrade confusion matrix with last value and after for loop we add the matrix and wrongs to dictionary and return confusion matrix so we can evaluate the precision ,recall and accuracy.

In [8]:
def calculate_save_and_print_info(Guess_amount_list,whole_info_el,seconds,len_Data):

    accuracy=round((Guess_amount_list.diagonal().sum()/len_Data)*100,4)
    precision=round(((Guess_amount_list.diagonal()/Guess_amount_list.sum(-1)).sum()/16)*100,4)
    recall=round(((Guess_amount_list.diagonal()/Guess_amount_list.sum(0)).sum()/16)*100,4)
    whole_info_el["Precision"]=precision
    whole_info_el["Accuracy"]=accuracy
    whole_info_el["Recall"]=recall
    print(f"Accuracy: {accuracy}%")
    print(f"Recall: {recall}%")
    print(f"Precision: {precision}%")
    print(f"Time past: {round(time.time()-seconds,2)} seconds")

    print("--------------------------------------------------------------------------------------------------------------\n")

This function takes a dictionary too and using the confusion matrix named as Guess_amount_list to find precision,recall,accuracy. For recall and accuracy it uses macro average to add them all.

In [9]:
def full_algorithm(data_to_read,k_amount,*all_ks):
    totaltime=time.time()
    
    whole_info={}

    data=read_input(data_to_read)
    


    for normalization in range(2):
        for k in all_ks:
                if normalization==1:
                        data=normalize(data)

                        
                        
                print("--------------------------------------------------------------------------------------------------------------")
                print(f"k constant: {k}\t Normalization: "+ (("Inactive "*(normalization==0) or "Active ")))
                seconds = time.time()



                whole_index=f"{k}, N:"+ ("Inactive"*(normalization==0) or "Active")

                whole_info[whole_index]={"Wrong_guesses":[],"Confusion_list":None,"Precision":0,"Accuracy":0,"Recall":0}
                whole_element=whole_info[whole_index]
                len_Data=len(data)
                after_knn=Knn(k,whole_element,k_amount,data,len_Data)
                calculate_save_and_print_info(after_knn,whole_element,seconds,len_Data)

    print(f"Total time: {round(time.time()-totaltime)}")

    return whole_info

this function's whole purpose is to return a nested dictionary that contains every information for every k and activation it does this by calling other functions. First it finds time to figure out total time then reads data named data_to_read parameter given by the user then there are 2 for loops first one for normalization and second for every k that was given by the user. It prints necesseary information and calls the functions above. Most import thing here is it creates a dictioonary and it creates a another dictionary inside this one named f"{k}, N:"+ ("Inactive"*(normalization==0) or "Active") and while calling functions it does not give the outer dictionary but it gives the one the inside so every function that adds something to this dictionary add it to the one named f"{k}, N:"+ ("Inactive"*(normalization==0) or "Active") so none of them overlaps.And after everything it returns this huge dictionary that contains every informaiton

In [10]:
whole_info=full_algorithm("16P.csv",5,1,3,5,7,9)

--------------------------------------------------------------------------------------------------------------
k constant: 1	 Normalization: Inactive 
Accuracy: 97.785%
Recall: 97.7914%
Precision: 97.7911%
Time past: 328.88 seconds
--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------
k constant: 3	 Normalization: Inactive 
Accuracy: 98.8616%
Recall: 98.8684%
Precision: 98.8685%
Time past: 342.06 seconds
--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------
k constant: 5	 Normalization: Inactive 
Accuracy: 98.9066%
Recall: 98.9133%
Precision: 98.9135%
Time past: 338.2 seconds
-------------------------------------------------------------------------------

This is the run code as you can see k in cross validation is 5 and we call this function for 1,3,5,7,9 values in KNN
here is the output of this code:
--------------------------------------------------------------------------------------------------------------
k constant: 1	 Normalization: Inactive 
Accuracy: 97.785%
Recall: 97.7914%
Precision: 97.7911%
Time past: 328.88 seconds
--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------
k constant: 3	 Normalization: Inactive 
Accuracy: 98.8616%
Recall: 98.8684%
Precision: 98.8685%
Time past: 342.06 seconds
--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------
k constant: 5	 Normalization: Inactive 
Accuracy: 98.9066%
Recall: 98.9133%
Precision: 98.9135%
Time past: 338.2 seconds
--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------
k constant: 7	 Normalization: Inactive 
Accuracy: 98.9333%
Recall: 98.94%
Precision: 98.9402%
Time past: 340.4 seconds
--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------
k constant: 9	 Normalization: Inactive 
Accuracy: 98.93%
Recall: 98.9366%
Precision: 98.9368%
Time past: 351.31 seconds
--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------
k constant: 1	 Normalization: Active 
Accuracy: 97.79%
Recall: 97.7963%
Precision: 97.7961%
Time past: 603.4 seconds
--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------
k constant: 3	 Normalization: Active 
Accuracy: 98.8683%
Recall: 98.8751%
Precision: 98.8752%
Time past: 592.46 seconds
--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------
k constant: 5	 Normalization: Active 
Accuracy: 98.9233%
Recall: 98.93%
Precision: 98.9302%
Time past: 619.33 seconds
--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------
k constant: 7	 Normalization: Active 
Accuracy: 98.93%
Recall: 98.9366%
Precision: 98.9368%
Time past: 628.93 seconds
--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------
k constant: 9	 Normalization: Active 
Accuracy: 98.9316%
Recall: 98.9383%
Precision: 98.9385%
Time past: 623.49 seconds
--------------------------------------------------------------------------------------------------------------



In [1]:
#ERROR FINDINGS
#first let's check some wrong guesses
for k in whole_info["5, N:Inactive"]["Confusion_list"]:
    print(list(k))#reason I turned it into list is because it was not able fit into the line when it was array
###output:
#[3720, 3, 4, 3, 3, 2, 1, 2, 3, 2, 1, 6, 2, 2, 1, 4]
#[1, 3705, 4, 3, 1, 2, 1, 2, 2, 3, 3, 1, 0, 3, 4, 2]
#[1, 2, 3707, 3, 3, 1, 6, 2, 1, 2, 2, 5, 1, 2, 5, 3]
#[2, 1, 4, 3707, 0, 1, 2, 4, 2, 1, 3, 0, 3, 5, 4, 3]
#[1, 4, 2, 0, 3712, 5, 3, 4, 0, 2, 1, 5, 3, 3, 6, 5]
#[5, 4, 2, 3, 1, 3701, 4, 1, 3, 2, 3, 1, 2, 1, 2, 4]
#[1, 2, 1, 1, 1, 3, 3708, 3, 5, 1, 2, 1, 4, 3, 1, 6]
#[3, 3, 2, 2, 0, 2, 2, 3721, 4, 5, 3, 1, 0, 0, 10, 3]
#[1, 4, 5, 1, 1, 3, 4, 1, 3714, 3, 1, 2, 2, 2, 1, 4]
#[4, 6, 3, 6, 2, 6, 5, 3, 4, 3713, 2, 1, 2, 3, 5, 4]
#[4, 2, 2, 2, 2, 2, 0, 4, 2, 3, 3718, 3, 3, 3, 5, 5]
#[4, 0, 4, 6, 3, 7, 5, 2, 1, 2, 2, 3713, 3, 4, 1, 3]
#[1, 4, 5, 5, 7, 4, 4, 1, 2, 3, 2, 1, 3705, 5, 2, 3]
#[1, 0, 5, 2, 3, 3, 4, 1, 2, 1, 1, 2, 2, 3708, 5, 5]
#[2, 3, 3, 5, 3, 4, 4, 2, 3, 3, 4, 3, 3, 2, 3694, 3]
#[1, 5, 3, 4, 4, 1, 1, 1, 1, 2, 3, 1, 3, 4, 3, 3697]
# As you can there is no 	flocculation at all almost every number other than true ones are  smaller than 5 so I think 
# the reason for 1 percent error is because data might be flawed little and also it might not be big enough maybe with
# a data set with 1 million rows would give a accuracy rate more than 99.9 percent

NameError: name 'whole_info' is not defined

error findings continues
If you look at the original output you would see that best Neighbor Number is 7 I guess it is because anything more than 7 causes high bias and lower variance so it turns out best is 5
One of the interesting thing was normalization effect was about 0.01 percent and it generally increased the accuracy so it was not a significant part f the project
I can't think of a reason my algorithm would have any effect on accuracy since what it does is completely mathmatical and there is no room for any chance but my algorithm is quite flexible since you can the data or the k values or cross validation values by just calling the function with different values
One of the other interesting things about this result is accuracy,recall and precision all have very similar values but this can be explained by looking at the confusion list again as you can see there is no flocculation so similarity between this values 